In [ ]:
import requests
import io
import glob
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tempfile, urllib, zipfile
import tensorflow_data_validation as tfdv

In [19]:
# Get the current folder thru print(os.getcwd())
#Download faa tables to /content

r = requests.get(url="https://registry.faa.gov/database/ReleasableAircraft.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("/content")

In [20]:
 arr = os.listdir('/content')
 print(arr)

['.config', 'DEALER.txt', 'ENGINE.txt', 'RESERVED.txt', 'ACFTREF.txt', 'DEREG.txt', 'MASTER.txt', 'ardata.pdf', 'DOCINDEX.txt', 'sample_data']


In [86]:
# Read MASTER.txt to Pandas dataFrame
df = pd.read_csv('/content/MASTER.txt', header=0, delimiter = ',', na_filter= True, encoding = 'utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [79]:
# Clean the dataset and split it to 2 (row 1 to row 280000 & row 280001 to the last row)
df = df.dropna(axis=1).dropna()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.replace(r'', np.NaN)
df1 = df.head(280000)
df2 = df.iloc[280001:]

In [80]:
df2.head()

,N-NUMBER,SERIAL NUMBER,MFR MDL CODE,ENG MFR MDL,YEAR MFR,TYPE REGISTRANT,NAME,STREET,STREET2,CITY,STATE,ZIP CODE,REGION,COUNTY,COUNTRY,LAST ACTION DATE,CERT ISSUE DATE,CERTIFICATION,TYPE ENGINE,STATUS CODE,MODE S CODE,FRACT OWNER,AIR WORTH DATE,OTHER NAMES(1),OTHER NAMES(2),OTHER NAMES(3),OTHER NAMES(4),OTHER NAMES(5),EXPIRATION DATE,UNIQUE ID,KIT MFR,KIT MODEL,MODE S CODE HEX
280001,971RD,40.508,2990002,41515,2005,1,MOUTON CHAD A,10126 PINE FLATS DR,NaN,HOUSTON,TX,770956660,2,201,US,20190530,20160405,1NU,1,V,53304327,NaN,20050428,NaN,NaN,NaN,NaN,NaN,20220430,173069,NaN,NaN,AD88D7
280002,971RH,797,1181511,03020,1972,3,REPUBLIC HELICOPTERS INC,8315 FM 2004,NaN,SANTA FE,TX,77510,2,167,US,20191014,19970305,1N,3,V,53304333,NaN,19831223,NaN,NaN,NaN,NaN,NaN,20230331,52944,NaN,NaN,AD88DB
280003,971RW,18267004,2072732,17026,1979,1,GRISCHKE ALAN E,3305 RIO DR,NaN,WESTON,WI,544766701,C,073,US,20190108,20130520,1N,1,V,53304350,NaN,19790524,NaN,NaN,NaN,NaN,NaN,20220531,64080,NaN,NaN,AD88E8
280004,971S,41213,060006W,55572,2013,1,COWDEN GARY L,1052 HOLIDAY LN,NaN,LYNCHBURG,VA,245044300,1,680,US,20191017,20191017,9A,8,V,53304354,NaN,20140310,NaN,NaN,NaN,NaN,NaN,20221031,1143689,NaN,NaN,AD88EC
280005,971SA,200022,9950100,41505,1996,3,SUCCESS AVIATION,503 MCKEEVER RD STE 1517,NaN,ARCOLA,TX,775832673,2,039,US,20191125,19970304,1U,1,V,53304355,NaN,19970508,NaN,NaN,NaN,NaN,NaN,20230430,66401,NaN,NaN,AD88ED


In [87]:
#Generate and visualize the Data Statistics
df1_stats = tfdv.generate_statistics_from_dataframe(df1)
df2_stats = tfdv.generate_statistics_from_dataframe(df2)
tfdv.visualize_statistics(lhs_statistics=df2_stats, rhs_statistics=df1_stats,
                          lhs_name='df2', rhs_name='df1')

In [85]:
#Infer a data schema for the first dataframe
schema = tfdv.infer_schema(df1_stats)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'N-NUMBER',BYTES,required,,-
'SERIAL NUMBER',BYTES,required,,-
'MFR MDL CODE',BYTES,required,,-
'ENG MFR MDL',BYTES,optional,single,-
'YEAR MFR',BYTES,optional,single,-
'TYPE REGISTRANT',BYTES,optional,single,-
'NAME',BYTES,optional,single,-
'STREET',BYTES,optional,single,-
'STREET2',BYTES,optional,single,-


/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'STATE',"'AE', 'AK', 'AL', 'AP', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'"
'REGION',"'1', '2', '3', '4', '5', '7', '8', 'C', 'E', 'S'"
'COUNTRY',"'AG', 'AI', 'AN', 'AR', 'AT', 'AU', 'BE', 'BG', 'BO', 'BS', 'BW', 'CA', 'CH', 'CN', 'CV', 'DE', 'DK', 'DO', 'ES', 'FM', 'FR', 'GB', 'GM', 'GR', 'GU', 'GY', 'HU', 'IE', 'IL', 'IT', 'JE', 'JP', 'KE', 'KY', 'LV', 'MP', 'MX', 'MY', 'NG', 'NL', 'NO', 'NZ', 'PA', 'PE', 'PH', 'PL', 'PT', 'PW', 'RQ', 'SA', 'SE', 'SG', 'TC', 'TH', 'TT', 'TW', 'UG', 'US', 'VC', 'VI', 'VU', 'ZA', 'ZW'"
'STATUS CODE',"'10', '11', '12', '17', '18', '19', '2', '20', '21', '22', '23', '25', '26', '27', '28', '29', '3', '4', '7', '9', 'D', 'M', 'N', 'R', 'V', 'W'"
'FRACT OWNER','Y'


In [13]:
#Detecting anomalies from the second dataframe 
anomalies = tfdv.validate_statistics(df2_stats, schema)
tfdv.display_anomalies(anomalies)

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'COUNTRY',Unexpected string values,Examples contain values missing from the schema: FI (<1%).
'SERIAL NUMBER',Invalid values,"String values that were not ints were found, such as ""E-2791""."


In [97]:
# Calculate data drift distance
skew_drift_anomalies = tfdv.validate_statistics(df1_stats, schema,serving_statistics=df2_stats)

# Display data drift
tfdv.display_anomalies(skew_drift_anomalies)

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'SERIAL NUMBER',High Linfty distance between training and serving,"The Linfty distance between training and serving is 0.0337903 (up to six significant digits), above the threshold 0.01. The feature value with maximum difference is: 001"
